### Python code for the paper
# Is there a negative trend in abundance of *Phyllopteryx taeniolatus* discernible in survey data summarised in Edgar et al. (2023)?
## Erik Schl&ouml;gl and David J. Booth
### Faculty of Science, University of Technology Sydney, Ultimo NSW 2007, Australia

In [1]:
# Load requisite packages
import pandas as pd
import datetime
import numpy as np
from xml.etree import ElementTree
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
# Python class to collect data by survey site or by 1-degree by 1-degree grid cell
class gridcell:
    def __init__(self,lat,lon,n):
        self.latitude = lat
        self.longitude = lon
        self.counts = np.zeros(n)
        self.totals = np.zeros(n)
    def __str__(self):
        return "Latitude:% s Longitude:% s" % (self.latitude, self.longitude)
    def __eq__(self,gc):
        if (not(self.latitude==gc.latitude)): 
            return False
        if (not(self.longitude==gc.longitude)): 
            return False
        return True
    def means(self):
        avg = [];
        for i in range(0,len(self.totals)):
            if (self.counts[i]>0):
                avg.append(self.totals[i]/self.counts[i])
            else:
                avg.append('NA')
        return avg
    def addvalues(self,valuelist):
        for i in range(0,len(valuelist)):
            if (valuelist[i]!='NA'):
                self.counts[i] += 1.0
                self.totals[i] += valuelist[i]

Read file of observation data line by line. Generate `gridcell` instances containing yearly observation data from `startyear` to `endyear`, pre-processed for conducting a Spearman's rank correlation test as described in Edgar et al. (2023) (i.e., in the same manner as implemented in their R code for this test).

In [18]:
def read_observations(fname,species,startyear,endyear):
    f = open(fname, "r")
    firstline = f.readline()
    # the first line gives the column labels - read this to determine the index of the columns we are interested in
    headerlist = firstline.split("|")
    #print(headerlist)
    scientific_nameidx = headerlist.index("species_name")
    start_idx = headerlist.index("x"+str(startyear))
    end_idx = headerlist.index("x"+str(endyear))
    lat_idx = headerlist.index("latitude")
    lon_idx = headerlist.index("longitude")
    #print(headerlist[start_idx:end_idx+1])
    n = len(headerlist)
    count = 0
    dflist = []
    gridcelllist = []
    while True:
        count += 1
        # Get next line from file
        line = f.readline()
        # if line is empty end of file is reached
        if (len(line)==0):
            break
        linelist = line.split("|")
        # if line has fewer columns than expected, consider the end of file reached
        if (len(linelist)<n):
            break
        if (linelist[scientific_nameidx]==species):
            has_data = False
            min_val = 10000000.0
            for x in linelist[start_idx:end_idx+1]:
                if (x!='NA'):
                    if (float(x)>0):
                        has_data = True
                        min_val = min(min_val,float(x))
            if (has_data):
                linelist.append(min_val)
                log_list = []
                summing = 0.0
                nsum = 0
                # calculate log values and set zeros to half min value
                for x in linelist[start_idx:end_idx+1]:
                    if (x!='NA'):
                        if (float(x)>0):
                            log_list.append(np.log(float(x)))
                        else:
                            log_list.append(np.log(min_val/2))
                        summing += log_list[-1]
                        nsum += 1
                    else:
                        log_list.append('NA')
                # standardise log values by subtracting mean
                for i in range(len(log_list)):
                    if (log_list[i]!='NA'):
                        log_list[i] -= summing/nsum
                linelist = linelist + log_list
                linelist.append(round(float(linelist[lat_idx])))
                linelist.append(round(float(linelist[lon_idx])))
                gc = gridcell(linelist[-2],linelist[-1],1+endyear-startyear)
                found_gc = False
                for i in range(0,len(gridcelllist)):
                    if (gridcelllist[i]==gc):
                        gridcelllist[i].addvalues(log_list)
                        found_gc = True
                if (not(found_gc)):
                    gc.addvalues(log_list)
                    gridcelllist.append(gc)
                #print(linelist)
                dflist.append(linelist)
    f.close()
    headerlist.append("Min value")
    for x in range(startyear,1+endyear):
        headerlist.append(x)
    headerlist.append("Rounded latitude")
    headerlist.append("Rounded longitude")
    #print(headerlist)
    df = pd.DataFrame(dflist,columns=headerlist)
    return df,gridcelllist

In [19]:
startyear,endyear = 2008,2021
df,gc_list = read_observations("Edgar_count_data.sep","Phyllopteryx taeniolatus",startyear,endyear)

In [8]:
# Write to CSV file for inspection of output so far
df.to_csv("Edgar_Phyllopteryx_taeniolatus.csv")

In [9]:
# grid cell means - note in particular the last two rows
for x in gc_list:
    print(x,x.means())

Latitude:-42 Longitude:148 ['NA', 'NA', 'NA', 0.3465735902799727, -0.3465735902799727, 0.3465735902799727, 'NA', 'NA', 0.3465735902799727, -0.3465735902799727, 'NA', -0.3465735902799727, 'NA', 'NA']
Latitude:-39 Longitude:146 ['NA', 'NA', -0.28881132523331066, 0.05776226504666204, 'NA', 'NA', 0.11552453009332408, 'NA', 'NA', 0.46209812037329656, 'NA', 'NA', -0.2310490601866486, 'NA']
Latitude:-38 Longitude:150 ['NA', -0.2310490601866486, 0.4620981203732968, 0.4620981203732968, 'NA', -0.2310490601866486, -0.2310490601866486, 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA']
Latitude:-36 Longitude:138 [0.5941261547656673, 'NA', 'NA', -0.09902102579427785, -0.09902102579427785, 'NA', -0.09902102579427785, 'NA', 'NA', 'NA', 'NA', -0.09902102579427785, -0.09902102579427785, -0.09902102579427785]
Latitude:-35 Longitude:151 [0.4371815415273663, 0.539114473768846, 0.38281515011999034, -0.1664909962140646, -0.2310490601866486, -0.07701635339555013, 'NA', -0.07701635339555013, -0.1664909962140646, -0.23

As noted in the <A HREF="https://scipy.github.io/devdocs/reference/generated/scipy.stats.spearmanr.html">SciPy documentation</A>, `stats.spearmanr()` uses an asymptotic approximation of the null distribution. Thus, "For small samples, it may be more appropriate to perform a permutation test." We implement this permutation test as suggested in the SciPy documentation.

In [16]:
def statistic(x):  # explore all possible pairings by permuting `x`
    rs = stats.spearmanr(x, y).statistic  # ignore pvalue
    transformed = rs * np.sqrt(dof / ((rs+1.0)*(1.0-rs)))
    return transformed

In [20]:
startyear,endyear = 2011,2021
df,gc_list = read_observations("Edgar_count_data.sep","Phyllopteryx taeniolatus",startyear,endyear)
total = gridcell(0,0,1+endyear-startyear)
for gc in gc_list:
    total.addvalues(gc.means())
x = range(startyear,1+endyear)
dof = len(x)-2
y = total.means()
print("Asymptotic test pvalue:",stats.spearmanr(x,y).pvalue)
ref = stats.permutation_test((x,), statistic, alternative='less',permutation_type='pairings',n_resamples=99990)
print("Permutation test pvalue:",ref.pvalue)

Asymptotic test pvalue: 0.4669223669820163
Permutation test pvalue: 0.2330509745877129


Running permutation test with ten times the number of permutations to check convergence:

In [21]:
ref = stats.permutation_test((x,), statistic, alternative='less',permutation_type='pairings',n_resamples=999900)
ref.pvalue

0.23447321284807196

In [22]:
startyear,endyear = 2010,2020
df,gc_list = read_observations("Edgar_count_data.sep","Phyllopteryx taeniolatus",startyear,endyear)
total = gridcell(0,0,1+endyear-startyear)
for gc in gc_list:
    total.addvalues(gc.means())
x = range(startyear,1+endyear)
dof = len(x)-2
y = total.means()
print("Asymptotic test pvalue:",stats.spearmanr(x,y).pvalue)
ref = stats.permutation_test((x,), statistic, alternative='less',permutation_type='pairings',n_resamples=99990)
print("Permutation test pvalue:",ref.pvalue)

Asymptotic test pvalue: 0.34029761418190363
Permutation test pvalue: 0.17126541388724986


In [23]:
startyear,endyear = 2009,2019
df,gc_list = read_observations("Edgar_count_data.sep","Phyllopteryx taeniolatus",startyear,endyear)
total = gridcell(0,0,1+endyear-startyear)
for gc in gc_list:
    total.addvalues(gc.means())
x = range(startyear,1+endyear)
dof = len(x)-2
y = total.means()
print("Asymptotic test pvalue:",stats.spearmanr(x,y).pvalue)
ref = stats.permutation_test((x,), statistic, alternative='less',permutation_type='pairings',n_resamples=99990)
print("Permutation test pvalue:",ref.pvalue)

Asymptotic test pvalue: 0.31182431390038623
Permutation test pvalue: 0.1561140502645238


In [24]:
startyear,endyear = 2008,2018
df,gc_list = read_observations("Edgar_count_data.sep","Phyllopteryx taeniolatus",startyear,endyear)
total = gridcell(0,0,1+endyear-startyear)
for gc in gc_list:
    total.addvalues(gc.means())
x = range(startyear,1+endyear)
dof = len(x)-2
y = total.means()
print("Asymptotic test pvalue:",stats.spearmanr(x,y).pvalue)
ref = stats.permutation_test((x,), statistic, alternative='less',permutation_type='pairings',n_resamples=99990)
print("Permutation test pvalue:",ref.pvalue)

Asymptotic test pvalue: 0.831716405381337
Permutation test pvalue: 0.4207078637077337


In [25]:
startyear,endyear = 2008,2021
df,gc_list = read_observations("Edgar_count_data.sep","Phyllopteryx taeniolatus",startyear,endyear)
total = gridcell(0,0,1+endyear-startyear)
for gc in gc_list:
    total.addvalues(gc.means())
x = range(startyear,1+endyear)
dof = len(x)-2
y = total.means()
print("Asymptotic test pvalue:",stats.spearmanr(x,y).pvalue)
ref = stats.permutation_test((x,), statistic, alternative='less',permutation_type='pairings',n_resamples=99990)
print("Permutation test pvalue:",ref.pvalue)

Asymptotic test pvalue: 0.06656042759911303
Permutation test pvalue: 0.035053154783930555


Extract the observation data to prepare for trend estimate.

In [26]:
dftrend = df[df.columns[11:41]]
dftrend

,x1992,x1993,x1994,x1995,x1996,x1997,x1998,x1999,x2000,x2001,...,x2012,x2013,x2014,x2015,x2016,x2017,x2018,x2019,x2020,x2021
0,NA,0,0,NA,NA,NA,NA,0,0,0,...,0,0.125,NA,NA,0.125,0,NA,0,NA,NA
1,NA,NA,NA,NA,NA,NA,NA,0,0,0,...,NA,NA,0.125,NA,NA,NA,NA,NA,NA,NA
2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,0,NA,NA,NA,NA,NA,NA,NA
3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,0,NA,0,NA,NA,NA,NA,0,0,0
4,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,0,NA,0,0,NA,0,0,NA,0
5,NA,NA,NA,NA,NA,NA,NA,NA,NA,0,...,0,NA,NA,NA,NA,0,NA,NA,NA,NA
6,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,0,NA,0,0,NA,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0.125,0,0,0,0,0,0,0,NA
8,0,0,0,0,0,0,0,0,0,0,...,NA,0,NA,NA,0,0,0,0,0,NA
9,0,0,0,0,0,0,0,0,0,0,...,0,0.125,0,0.125,0,0,0.125,0,0,NA


Interpolate and extrapolate as in Edgar et al. (2023) to eliminate `'NA'` values:

In [27]:
for row in range(0,dftrend.shape[0]):
    tmp = dftrend.iloc[row].values
    prevval = -1.0
    for i in range(0,len(tmp)):
        j = i
        if (tmp[i]=='NA'):
            while ((j<len(tmp)and(tmp[j]=='NA'))):
                j = j+1
            if (j<len(tmp)):
                if (prevval==-1):
                    for k in range(i,j):
                        tmp[k] = float(tmp[j])
                else:
                    for k in range(i,j):
                        tmp[k] = prevval + (k+1-i)/(j+1-i)*(float(tmp[j])-prevval)
                prevval = float(tmp[j])
                i = j
                tmp[i] = float(tmp[i])
            else:
                for k in range(i,j):
                    tmp[k] = prevval
                i = len(tmp)
        else: 
            tmp[i] = float(tmp[i])
            prevval = tmp[i]
dftrend

,x1992,x1993,x1994,x1995,x1996,x1997,x1998,x1999,x2000,x2001,...,x2012,x2013,x2014,x2015,x2016,x2017,x2018,x2019,x2020,x2021
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.125,0.125,0.125,0.125,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041667,0.083333,0.125,0.125,0.125,0.125,0.125,0.125,0.125,0.125
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0625,0.03125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,0.083333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.125,0.0,0.125,0.0,0.0,0.125,0.0,0.0,0.0


Aggregate in 1-degree by 1-degree grid cells and average as in Edgar et al. (2023):

In [28]:
def dfread_observations(df,dftrend):
    gridcelllist = []
    for row in range(0,dftrend.shape[0]):
        gc = gridcell(df['Rounded latitude'][row],df['Rounded latitude'][row],dftrend.shape[1])
        found_gc = False
        for i in range(0,len(gridcelllist)):
            if (gridcelllist[i]==gc):
                gridcelllist[i].addvalues(dftrend.iloc[row].values)
                found_gc = True
        if (not(found_gc)):
            gc.addvalues(dftrend.iloc[row].values)
            gridcelllist.append(gc)
    return gridcelllist

In [29]:
gclist = dfread_observations(df,dftrend)
len(gclist)

9

In [30]:
gcsum = np.array(gclist[0].means())
for gc in gclist[1:]:
    gcsum += gc.means()
gcmean = gcsum/len(gclist)

Calculate natural logarithms of yearly averages:

In [31]:
loggcmean = np.log(gcmean)
loggcmean

array([-2.71123815, -2.71123815, -2.69241652, -2.70965608, -2.70807651,
       -2.70649943, -2.70492483, -2.70335271, -2.67904012, -2.70021586,
       -2.69865112, -2.69708882, -2.69552896, -2.75162592, -2.72078349,
       -2.72384173, -2.58667819, -2.84100595, -2.68906752, -2.69116331,
       -2.91846771, -2.78818858, -2.95016189, -2.69690957, -2.46857571,
       -2.82226052, -2.84207857, -3.11130815, -3.1984627 , -3.21404743])

In [32]:
import scipy.stats as stats

Perform the trendline regression for Table 1, i.e., estimated trendlines for different data windows, all ending in 2021, but starting in different years:

In [34]:
years = []
start = []
end   = []
change = []
tbl1df = pd.DataFrame()
for i in range(3,21):
    result = stats.linregress(np.array([x for x in range(0,i)]),loggcmean[-i:])
    beta = result.slope
    ic = result.intercept
    sv,ev = np.exp(ic),np.exp(ic+(i-1)*beta)
    years.append(i)
    start.append(2022-i)
    end.append(2021)
    change.append("{0:0.1f}".format(100*(ev/sv-1)))
tbl1df["Years"] = years
tbl1df["Start"] = start
tbl1df["End"]   = end
tbl1df["% change"] = change
tbl1df

,Years,Start,End,% change
0,3,2019,2021,-9.8
1,4,2018,2021,-30.3
2,5,2017,2021,-36.6
3,6,2016,2021,-51.9
4,7,2015,2021,-50.7
5,8,2014,2021,-40.9
6,9,2013,2021,-38.5
7,10,2012,2021,-32.2
8,11,2011,2021,-34.3
9,12,2010,2021,-35.4


Output table in LaTeX for paper:

In [35]:
print("\\begin{tabular}{|r|c|c|r|}\n\\hline")
print("Years & Start & End & \\% change\\\\ \n\\hline")
for i in range(3,21):
    result = stats.linregress(np.array([x for x in range(0,i)]),loggcmean[-i:])
    beta = result.slope
    ic = result.intercept
    sv,ev = np.exp(ic),np.exp(ic+(i-1)*beta)
    #print(i,2022-i,2021,sv,ev,ev/sv-1)
    print(i,'&',2022-i,'&',2021,'&',"{0:0.1f}".format(100*(ev/sv-1)),"\\\\ \\hline")
print("\\end{tabular}")

\begin{tabular}{|r|c|c|r|}
\hline
Years & Start & End & \% change\\ 
\hline
3 & 2019 & 2021 & -9.8 \\ \hline
4 & 2018 & 2021 & -30.3 \\ \hline
5 & 2017 & 2021 & -36.6 \\ \hline
6 & 2016 & 2021 & -51.9 \\ \hline
7 & 2015 & 2021 & -50.7 \\ \hline
8 & 2014 & 2021 & -40.9 \\ \hline
9 & 2013 & 2021 & -38.5 \\ \hline
10 & 2012 & 2021 & -32.2 \\ \hline
11 & 2011 & 2021 & -34.3 \\ \hline
12 & 2010 & 2021 & -35.4 \\ \hline
13 & 2009 & 2021 & -31.9 \\ \hline
14 & 2008 & 2021 & -35.3 \\ \hline
15 & 2007 & 2021 & -34.5 \\ \hline
16 & 2006 & 2021 & -33.8 \\ \hline
17 & 2005 & 2021 & -32.3 \\ \hline
18 & 2004 & 2021 & -32.1 \\ \hline
19 & 2003 & 2021 & -31.8 \\ \hline
20 & 2002 & 2021 & -31.4 \\ \hline
\end{tabular}


Perform the trendline regression for Table 2, i.e., estimated trendlines for different data windows, ending in 2017, 2018, 2019 and 2020, and starting in different years:

In [36]:
startlist = []
change2017 = []
change2018 = []
change2019 = []
change2020 = []
changelist = [change2020,change2019,change2018,change2017]
tbl2df = pd.DataFrame()
for i in range(3,12):
    start = 2018-i
    startlist.append(start)
    for offset in [4,3,2,1]:
        datlen = i + 4 - offset
        result = stats.linregress(np.array([x for x in range(0,datlen)]),loggcmean[-datlen-offset:-offset])
        beta = result.slope
        ic = result.intercept
        sv,ev = np.exp(ic),np.exp(ic+(datlen-1)*beta)
        changelist[offset-1].append("{0:0.1f}".format(100*(ev/sv-1)))
tbl2df["Start"] = startlist
tbl2df["% change to 2017"] = change2017
tbl2df["% change to 2018"] = change2018
tbl2df["% change to 2019"] = change2019
tbl2df["% change to 2020"] = change2020
tbl2df

,Start,% change to 2017,% change to 2018,% change to 2019,% change to 2020
0,2015,-11.8,-21.1,-38.2,-47.1
1,2014,20.2,3.7,-20.4,-34.1
2,2013,18.0,5.0,-17.2,-31.1
3,2012,27.4,14.8,-8.1,-23.4
4,2011,13.7,5.4,-12.9,-26.2
5,2010,5.9,-0.2,-15.8,-27.8
6,2009,9.7,3.7,-11.7,-23.9
7,2008,-1.1,-5.2,-17.8,-28.3
8,2007,-1.9,-5.6,-17.5,-27.5


Output table in LaTeX for paper:

In [38]:
print("\\begin{tabular}{|c|r|r|r|r|}\n\\hline")
print("Start & \\% change to 2017 & \\% change to 2018 & \\% change to 2019 & \\% change to 2020 \\\\ \n\\hline")
for i in range(3,12):
    start = 2018-i
    print(start)
    for offset in [4,3,2,1]:
        datlen = i + 4 - offset
        result = stats.linregress(np.array([x for x in range(0,datlen)]),loggcmean[-datlen-offset:-offset])
        beta = result.slope
        ic = result.intercept
        sv,ev = np.exp(ic),np.exp(ic+(datlen-1)*beta)
        print('&',"{0:0.1f}".format(100*(ev/sv-1)))
    print("\\\\ \\hline")
print("\\end{tabular}")

\begin{tabular}{|c|r|r|r|r|}
\hline
Start & \% change to 2017 & \% change to 2018 & \% change to 2019 & \% change to 2020 \\ 
\hline
2015
& -11.8
& -21.1
& -38.2
& -47.1
\\ \hline
2014
& 20.2
& 3.7
& -20.4
& -34.1
\\ \hline
2013
& 18.0
& 5.0
& -17.2
& -31.1
\\ \hline
2012
& 27.4
& 14.8
& -8.1
& -23.4
\\ \hline
2011
& 13.7
& 5.4
& -12.9
& -26.2
\\ \hline
2010
& 5.9
& -0.2
& -15.8
& -27.8
\\ \hline
2009
& 9.7
& 3.7
& -11.7
& -23.9
\\ \hline
2008
& -1.1
& -5.2
& -17.8
& -28.3
\\ \hline
2007
& -1.9
& -5.6
& -17.5
& -27.5
\\ \hline
\end{tabular}
